In [3]:
#Gabor filter n1
import os
import cv2
from PIL import Image
import numpy as np

def apply_gabor(image):
    # Ensure the image is in grayscale for Gabor processing
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gabor_kernel = cv2.getGaborKernel((21, 21), 8, np.pi/4, 10, 0.5, 0, ktype=cv2.CV_32F)
    filtered_image = cv2.filter2D(image, -1, gabor_kernel)
    return filtered_image

def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created directory: {directory}")

def process_and_save_images(source_dir, target_dir):
    categories = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']
    for category in categories:
        source_path = os.path.join(source_dir, category)
        target_path = os.path.join(target_dir, category)
        
        # Check if the source directory exists before processing
        if os.path.exists(source_path):
            ensure_dir(target_path)
            images = os.listdir(source_path)
            if images:
                for image_name in images:
                    image_path = os.path.join(source_path, image_name)
                    try:
                        image = cv2.imread(image_path)
                        if image is not None:
                            image = apply_gabor(image)
                            # Convert back to PIL image to save in directory
                            image = Image.fromarray(image)
                            image.save(os.path.join(target_path, image_name))
                    except Exception as e:
                        print(f"Error processing {image_path}: {str(e)}")
            else:
                print(f"No images found in {source_path}")
        else:
            print(f"Directory does not exist: {source_path}")

# Define your base and target directories
base_dir = 'Alzheimer_s Dataset'  # Adjust this to your dataset path
target_base_dir = 'alzheimer_gabor'

# Process train and test datasets
process_and_save_images(os.path.join(base_dir, 'train'), os.path.join(target_base_dir, 'train'))
process_and_save_images(os.path.join(base_dir, 'test'), os.path.join(target_base_dir, 'test'))


Created directory: alzheimer_gabor/train/MildDemented
Created directory: alzheimer_gabor/train/ModerateDemented
Created directory: alzheimer_gabor/train/NonDemented
Created directory: alzheimer_gabor/train/VeryMildDemented
Created directory: alzheimer_gabor/test/MildDemented
Created directory: alzheimer_gabor/test/ModerateDemented
Created directory: alzheimer_gabor/test/NonDemented
Created directory: alzheimer_gabor/test/VeryMildDemented


In [4]:
#Gabor filter n2
import os
import cv2
from PIL import Image
import numpy as np

def create_gabor_filters():
    filters = []
    ksize = 21
    # Define different thetas for different orientations
    thetas = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    # Define different lambdas for different wavelengths
    lambdas = [10, 20]
    for theta in thetas:
        for lambd in lambdas:
            gabor_kernel = cv2.getGaborKernel((ksize, ksize), 4.0, theta, lambd, 0.5, 0, ktype=cv2.CV_32F)
            filters.append(gabor_kernel)
    return filters

def apply_gabor(image, filters):
    # Ensure the image is in grayscale
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # Apply each filter
    filtered_images = []
    for i, gabor_kernel in enumerate(filters):
        filtered_image = cv2.filter2D(image, -1, gabor_kernel)
        filtered_images.append(filtered_image)
    return filtered_images

def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created directory: {directory}")

def process_and_save_images(source_dir, target_dir, filters):
    categories = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']
    for category in categories:
        source_path = os.path.join(source_dir, category)
        target_path = os.path.join(target_dir, category)
        if os.path.exists(source_path):
            ensure_dir(target_path)
            images = os.listdir(source_path)
            for image_name in images:
                image_path = os.path.join(source_path, image_name)
                image = cv2.imread(image_path)
                if image is not None:
                    filtered_images = apply_gabor(image, filters)
                    for idx, filtered_image in enumerate(filtered_images):
                        # Generate a unique filename for each filtered image
                        filename = f"{os.path.splitext(image_name)[0]}_filter{idx}.png"
                        image = Image.fromarray(filtered_image)
                        image.save(os.path.join(target_path, filename))
                else:
                    print(f"Failed to read image: {image_path}")

# Initialize Gabor filters
filters = create_gabor_filters()

# Define your base and target directories
base_dir = 'Alzheimer_s Dataset'  # Adjust this to your dataset path
target_base_dir = 'alzheimer_gabor_8'

# Process train and test datasets
process_and_save_images(os.path.join(base_dir, 'train'), os.path.join(target_base_dir, 'train'), filters)
process_and_save_images(os.path.join(base_dir, 'test'), os.path.join(target_base_dir, 'test'), filters)


Created directory: alzheimer_gabor_8/train/MildDemented
Created directory: alzheimer_gabor_8/train/ModerateDemented
Created directory: alzheimer_gabor_8/train/NonDemented
Created directory: alzheimer_gabor_8/train/VeryMildDemented
Created directory: alzheimer_gabor_8/test/MildDemented
Failed to read image: Alzheimer_s Dataset/test/MildDemented/.ipynb_checkpoints
Created directory: alzheimer_gabor_8/test/ModerateDemented
Failed to read image: Alzheimer_s Dataset/test/ModerateDemented/.ipynb_checkpoints
Created directory: alzheimer_gabor_8/test/NonDemented
Created directory: alzheimer_gabor_8/test/VeryMildDemented
Failed to read image: Alzheimer_s Dataset/test/VeryMildDemented/.ipynb_checkpoints


In [ ]:
def sobel_edge_detection(image):
    """
    Applies the Sobel edge detection algorithm to an input image.

    Args:
    image: A tensor of shape (height, width, channels), representing the input image.

    Returns:
    A tensor of the same shape as the input, containing the edge-detected image.
    """
    # Add a batch dimension if it's missing
    if len(image.shape) == 3:
        image = tf.expand_dims(image, axis=0)

    # Define Sobel filters for x and y directions.
    sobel_x = tf.constant([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=tf.float32)
    sobel_x_filter = tf.reshape(sobel_x, [3, 3, 1, 1])
    sobel_y = tf.constant([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=tf.float32)
    sobel_y_filter = tf.reshape(sobel_y, [3, 3, 1, 1])

    # Convert the image to grayscale if it has more than one channel.
    if image.shape[-1] > 1:
        image = tf.image.rgb_to_grayscale(image)

    # Filter the image with Sobel kernels.
    Gx = tf.nn.conv2d(input=image, filters=sobel_x_filter, strides=[1, 1, 1, 1], padding='SAME')
    Gy = tf.nn.conv2d(input=image, filters=sobel_y_filter, strides=[1, 1, 1, 1], padding='SAME')

    # Compute the magnitude of the gradients.
    G = tf.sqrt(tf.square(Gx) + tf.square(Gy))
    G = tf.squeeze(G, axis=0)  # Remove the batch dimension.

    return G